In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Rikitea,PF,2021-06-06 20:17:35,-23.1203,-134.9692,76.86,84,96,15.68
1,1,Tual,ID,2021-06-06 20:15:58,-5.6667,132.7500,80.91,78,100,14.32
2,2,Amga,RU,2021-06-06 20:17:36,60.8953,131.9608,60.26,77,100,3.76
3,3,Bredasdorp,ZA,2021-06-06 20:14:52,-34.5322,20.0403,59.45,32,5,6.35
4,4,Guerrero Negro,MX,2021-06-06 20:17:36,27.9769,-114.0611,72.81,53,84,12.95


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [11]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Rikitea,PF,2021-06-06 20:17:35,-23.1203,-134.9692,76.86,84,96,15.68
1,1,Tual,ID,2021-06-06 20:15:58,-5.6667,132.7500,80.91,78,100,14.32
9,9,Kapaa,US,2021-06-06 20:17:37,22.0752,-159.3190,82.38,82,90,1.99
10,10,Oistins,BB,2021-06-06 20:16:58,13.0667,-59.5333,86.09,58,40,20.71
11,11,Trincomalee,LK,2021-06-06 20:17:38,8.5711,81.2335,81.28,77,100,18.50
17,17,Barreiras,BR,2021-06-06 20:17:39,-12.1528,-44.9900,86.81,26,71,8.23
23,23,Auki,SB,2021-06-06 20:17:41,-8.7676,160.7034,77.63,86,10,3.53
32,32,Rapid Valley,US,2021-06-06 20:17:43,44.0625,-103.1463,80.04,34,1,1.99
36,36,Salinopolis,BR,2021-06-06 20:17:43,-0.6136,-47.3561,80.98,82,61,9.06
38,38,Jaru,BR,2021-06-06 20:17:44,-10.4389,-62.4664,87.64,60,100,2.13


In [15]:
preferred_cities_df.count()

City_ID       166
City          166
Country       164
Date          166
Lat           166
Lng           166
Max Temp      166
Humidity      166
Cloudiness    166
Wind Speed    166
dtype: int64

In [21]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Rikitea,PF,76.86,-23.1203,-134.9692,
1,Tual,ID,80.91,-5.6667,132.7500,
9,Kapaa,US,82.38,22.0752,-159.3190,
10,Oistins,BB,86.09,13.0667,-59.5333,
11,Trincomalee,LK,81.28,8.5711,81.2335,
17,Barreiras,BR,86.81,-12.1528,-44.9900,
23,Auki,SB,77.63,-8.7676,160.7034,
32,Rapid Valley,US,80.04,44.0625,-103.1463,
36,Salinopolis,BR,80.98,-0.6136,-47.3561,
38,Jaru,BR,87.64,-10.4389,-62.4664,


In [22]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    # Grab the first hotel from the results and store the name.
    try:
      hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
      print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]